**To run locally:**
1. `pip install keras`
2. `pip install tensorflow`
3. cd into folder and run `jupyter notebook`

In [23]:
from keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [24]:
x_train.shape

(60000, 28, 28)

In [25]:
import keras; print(keras.__version__);

2.1.6


In [26]:
import numpy as np
np.random.seed(123) 

In [27]:
from matplotlib import pyplot as plt
plt.imshow(x_train[0])

In [28]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')

In [29]:
x_train.shape

(60000, 28, 28, 1)

In [30]:
x_train /= 255
x_test /= 255

In [31]:
y_train[:10]

array([9, 0, 0, 3, 0, 2, 7, 2, 5, 5], dtype=uint8)

In [32]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [33]:
# First 10 results
y_train[:10]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]], dtype=float32)

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [35]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [36]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [37]:
model.fit(x_train, y_train, 
          batch_size=32, nb_epoch=10, verbose=1)

Epoch 1/10
60000/60000 [==============================] - 106s 2ms/step - loss: 0.5028 - acc: 0.8144
Epoch 2/10
60000/60000 [==============================] - 94s 2ms/step - loss: 0.3502 - acc: 0.8693
Epoch 3/10
60000/60000 [==============================] - 106s 2ms/step - loss: 0.3128 - acc: 0.8841
Epoch 4/10
60000/60000 [==============================] - 103s 2ms/step - loss: 0.2838 - acc: 0.8926
Epoch 5/10
60000/60000 [==============================] - 105s 2ms/step - loss: 0.2689 - acc: 0.8987
Epoch 6/10
60000/60000 [==============================] - 103s 2ms/step - loss: 0.2568 - acc: 0.9027
Epoch 7/10
60000/60000 [==============================] - 110s 2ms/step - loss: 0.2457 - acc: 0.9069
Epoch 8/10
60000/60000 [==============================] - 114s 2ms/step - loss: 0.2387 - acc: 0.9103
Epoch 9/10
60000/60000 [==============================] - 113s 2ms/step - loss: 0.2285 - acc: 0.9130
Epoch 10/10
60000/60000 [==============================] - 108s 2ms/step - loss: 0.2225 - ac

In [38]:
score = model.evaluate(x_test, y_test, verbose=1)

10000/10000 [==============================] - 4s 359us/step


In [39]:
score

[0.23539057997465135, 0.9138]

In [40]:
model.save('my_model.h5')

# **Testing hyper parameters**

In [41]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

We will be testing batch size and epochs

In [42]:
batch_size = [10, 32, 100]
epochs = [10, 15, 20]
param_grid = dict(batch_size=batch_size, epochs=epochs)

In the interest of time, we will test hyper parameters on a small sample (10%)

In [43]:
x_train_sample = x_train[:6000]
y_train_sample = y_train[:6000]

create_model method to generate model

In [44]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu', input_shape=(28,28,1)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

We create a grid of our hyper parameters and create models against all of them.

In [107]:
model = KerasClassifier(build_fn=create_model, verbose=1)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(x_train_sample, y_train_sample)

Epoch 1/10
4000/4000 [==============================] - 24s 6ms/step - loss: 0.8656 - acc: 0.6795
Epoch 2/10
4000/4000 [==============================] - 17s 4ms/step - loss: 0.5716 - acc: 0.7880
Epoch 3/10
4000/4000 [==============================] - 20s 5ms/step - loss: 0.5056 - acc: 0.8125
Epoch 4/10
4000/4000 [==============================] - 17s 4ms/step - loss: 0.4482 - acc: 0.8317
Epoch 5/10
4000/4000 [==============================] - 15s 4ms/step - loss: 0.4149 - acc: 0.8440
Epoch 6/10
4000/4000 [==============================] - 17s 4ms/step - loss: 0.3757 - acc: 0.8550
Epoch 7/10
4000/4000 [==============================] - 17s 4ms/step - loss: 0.3469 - acc: 0.8695
Epoch 8/10
4000/4000 [==============================] - 17s 4ms/step - loss: 0.3150 - acc: 0.8767
Epoch 9/10
4000/4000 [==============================] - 17s 4ms/step - loss: 0.3155 - acc: 0.8747
Epoch 10/10
4000/4000 [==============================] - 3s 709us/step
Epoch 1/10
4000/4000 [=========================

4000/4000 [==============================] - 9s 2ms/step - loss: 0.3563 - acc: 0.8660
Epoch 9/10
4000/4000 [==============================] - 9s 2ms/step - loss: 0.3410 - acc: 0.8745
Epoch 10/10
4000/4000 [==============================] - 2s 503us/step
Epoch 1/10
4000/4000 [==============================] - 16s 4ms/step - loss: 0.9800 - acc: 0.6372: 1s - loss: 1.024
Epoch 2/10
4000/4000 [==============================] - 11s 3ms/step - loss: 0.6461 - acc: 0.7580
Epoch 3/10
4000/4000 [==============================] - 8s 2ms/step - loss: 0.5498 - acc: 0.7995
Epoch 4/10
4000/4000 [==============================] - 9s 2ms/step - loss: 0.5025 - acc: 0.8097
Epoch 5/10
4000/4000 [==============================] - 11s 3ms/step - loss: 0.4675 - acc: 0.8287
Epoch 6/10
4000/4000 [==============================] - 9s 2ms/step - loss: 0.4496 - acc: 0.8357
Epoch 7/10
4000/4000 [==============================] - 9s 2ms/step - loss: 0.4102 - acc: 0.8420
Epoch 8/10
4000/4000 [========================

4000/4000 [==============================] - 1s 350us/step
Epoch 1/20
4000/4000 [==============================] - 14s 4ms/step - loss: 0.9048 - acc: 0.6755
Epoch 2/20
4000/4000 [==============================] - 8s 2ms/step - loss: 0.5850 - acc: 0.7768
Epoch 3/20
4000/4000 [==============================] - 8s 2ms/step - loss: 0.5201 - acc: 0.8075
Epoch 4/20
4000/4000 [==============================] - 7s 2ms/step - loss: 0.4518 - acc: 0.8325
Epoch 5/20
4000/4000 [==============================] - 8s 2ms/step - loss: 0.4248 - acc: 0.8427A: 0s - loss: 0.4218 -
Epoch 6/20
4000/4000 [==============================] - 7s 2ms/step - loss: 0.4013 - acc: 0.8515
Epoch 7/20
4000/4000 [==============================] - 7s 2ms/step - loss: 0.3606 - acc: 0.8695
Epoch 8/20
4000/4000 [==============================] - 7s 2ms/step - loss: 0.3391 - acc: 0.8740
Epoch 9/20
4000/4000 [==============================] - 8s 2ms/step - loss: 0.3231 - acc: 0.8692
Epoch 10/20
4000/4000 [======================

4000/4000 [==============================] - 5s 1ms/step - loss: 0.4846 - acc: 0.8190
Epoch 7/15
4000/4000 [==============================] - 5s 1ms/step - loss: 0.4457 - acc: 0.8368
Epoch 8/15
4000/4000 [==============================] - 5s 1ms/step - loss: 0.4445 - acc: 0.8375
Epoch 9/15
4000/4000 [==============================] - 5s 1ms/step - loss: 0.4247 - acc: 0.8377
Epoch 10/15
4000/4000 [==============================] - 6s 1ms/step - loss: 0.4014 - acc: 0.8550
Epoch 11/15
4000/4000 [==============================] - 5s 1ms/step - loss: 0.3899 - acc: 0.8502
Epoch 12/15
4000/4000 [==============================] - 5s 1ms/step - loss: 0.3614 - acc: 0.8690
Epoch 13/15
4000/4000 [==============================] - 5s 1ms/step - loss: 0.3577 - acc: 0.8670
Epoch 14/15
4000/4000 [==============================] - 6s 1ms/step - loss: 0.3267 - acc: 0.8805
Epoch 15/15
4000/4000 [==============================] - 1s 264us/step
Epoch 1/15
4000/4000 [==============================] - 10s 3m

Epoch 20/20
4000/4000 [==============================] - 1s 367us/step
Epoch 1/20
4000/4000 [==============================] - 11s 3ms/step - loss: 1.2450 - acc: 0.5475
Epoch 2/20
4000/4000 [==============================] - 5s 1ms/step - loss: 0.7159 - acc: 0.7297
Epoch 3/20
4000/4000 [==============================] - 5s 1ms/step - loss: 0.6172 - acc: 0.7722
Epoch 4/20
4000/4000 [==============================] - 5s 1ms/step - loss: 0.5634 - acc: 0.7907
Epoch 5/20
4000/4000 [==============================] - 5s 1ms/step - loss: 0.5581 - acc: 0.7910
Epoch 6/20
4000/4000 [==============================] - 5s 1ms/step - loss: 0.5309 - acc: 0.8010
Epoch 7/20
4000/4000 [==============================] - 5s 1ms/step - loss: 0.4917 - acc: 0.8205
Epoch 8/20
4000/4000 [==============================] - 5s 1ms/step - loss: 0.4658 - acc: 0.8242
Epoch 9/20
4000/4000 [==============================] - 5s 1ms/step - loss: 0.4552 - acc: 0.8320
Epoch 10/20
4000/4000 [==============================] 

In [111]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.870833 using {'epochs': 20, 'batch_size': 32}
0.862833 (0.008024) with: {'epochs': 10, 'batch_size': 10}
0.855000 (0.009626) with: {'epochs': 15, 'batch_size': 10}
0.861833 (0.007760) with: {'epochs': 20, 'batch_size': 10}
0.856667 (0.011813) with: {'epochs': 10, 'batch_size': 32}
0.866833 (0.003064) with: {'epochs': 15, 'batch_size': 32}
0.870833 (0.006562) with: {'epochs': 20, 'batch_size': 32}
0.839500 (0.010638) with: {'epochs': 10, 'batch_size': 100}
0.859667 (0.008290) with: {'epochs': 15, 'batch_size': 100}
0.861667 (0.006355) with: {'epochs': 20, 'batch_size': 100}


# Build model with batch size 32, epochs 20

In [45]:
final_model = create_model()
final_model.compile(loss='categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])
final_model.fit(x_train, y_train,
                batch_size=32, nb_epoch=20, verbose=1)
final_score = final_model.evaluate(x_test, y_test, verbose=1)

Epoch 1/20
60000/60000 [==============================] - 108s 2ms/step - loss: 0.5102 - acc: 0.8098
Epoch 2/20
60000/60000 [==============================] - 107s 2ms/step - loss: 0.3571 - acc: 0.8666
Epoch 3/20
60000/60000 [==============================] - 109s 2ms/step - loss: 0.3190 - acc: 0.8813
Epoch 4/20
60000/60000 [==============================] - 109s 2ms/step - loss: 0.2915 - acc: 0.8906
Epoch 5/20
60000/60000 [==============================] - 109s 2ms/step - loss: 0.2775 - acc: 0.8954
Epoch 6/20
60000/60000 [==============================] - 108s 2ms/step - loss: 0.2621 - acc: 0.9009
Epoch 7/20
60000/60000 [==============================] - 110s 2ms/step - loss: 0.2496 - acc: 0.9051
Epoch 8/20
60000/60000 [==============================] - 110s 2ms/step - loss: 0.2392 - acc: 0.9086
Epoch 9/20
60000/60000 [==============================] - 110s 2ms/step - loss: 0.2311 - acc: 0.9126
Epoch 10/20
60000/60000 [==============================] - 110s 2ms/step - loss: 0.2254 - a

NameError: name 'final_store' is not defined

In [46]:
final_score

[0.2498152796149254, 0.916]